In [1]:
import requests
import urllib
import time
import copy
import re
from bs4 import BeautifulSoup

In [2]:
#1 Part
#1.1 Section
#1.1.1 Subsection
#1.1.1.1. Article
#	(1) Sentence
#		(a) Clause
#			(i) Subclause


#Actual dictionary structure looks like this:
#ArtX
    #ArticleX
    #SentenceX
        #ClauseX
            #SubclauseX

url = 'https://www.ontario.ca/laws/regulation/120332'
response = requests.get(url)
print(response)
soup = BeautifulSoup(response.text, "html.parser")

<Response [200]>


In [3]:
#Make list of all the results for 'p' tags
#notice limit is just so we have a smaller list to test with
pTags = soup.find_all('p', class_ = ['section-e', 'subsection-e', 'clause-e', 'subclause-e'], limit = 30)

In [4]:
#create dictionary to contain dictionaries
p: dict = {}

#index for articles (for now), needs different regex expression than sentences, etc.
article = -1

In [118]:
def formatText(text): 
    
        text = str(text)
    
        cleaned = re.sub(r"<b>|</b>", "", text)
        cleaned = re.sub(r"<i>|</i>", "", cleaned)
        
        if (">" in cleaned):
              
            start = cleaned.index(">") + 1
            cleaned = cleaned[start:]
            
        if ("<" in cleaned):
        
            end = cleaned.index("<")
            cleaned = cleaned[:end]
        

        return cleaned

In [119]:
def cleanText(list): 
    
    cleaned = []
    
    for i in list:
        clean_text = formatText(i)
        cleaned.append(clean_text)
        
    return cleaned

In [120]:
cleanText(pTags)

['1.1.1.1. Scope of Division A',
 '  (1) Division A contains compliance and application provisions and the objectives and functional statements\r\nof this Code.',
 '1.1.1.2. Scope of Division B',
 '  (1) Division B contains the acceptable solutions\r\nof this Code.',
 '1.1.1.3. Scope of Division C',
 '  (1) Division C contains the administrative provisions of this Code.',
 '1.1.1.4. Internal Cross-references',
 '  (1) If a provision of this Code contains a reference to another provision of this Code but no Division is specified, both provisions are in the same Division of this Code.',
 '1.1.2.1. Application of Parts 1, 7 and 12',
 ' (1) Part 1 of Division B applies to all buildings.',
 ' (2) Subject to Article 1.1.2.6., Parts 7 and 12 of Division B apply to all buildings.',
 '1.1.2.2. Application of Parts 3, 4, 5 and 6',
 ' (1) Subject to Articles 1.1.2.6. and 1.3.1.2., Parts 3, 5 and 6 of Division B apply to all buildings,',
 '  (a) used for major occupancies classified as,',
 '  (i) 

In [121]:
def breakPoints(list):
    
    points = []
    previous_index = 0
    
    for i in list:
        
        if i.attrs['class'] == ['section-e']:
            
            title = formatText(i)
            index = list.index(i)
            
            
            points.append({"title": title, "starting index": index})
            
            
    for i in points:
        
        index = points.index(i)
        
        if (index == len(points) - 1):
            
            i["ending index"] = index
            break
            
        else:
        
            i["ending index"] = points[index + 1]["starting index"]
        
        
    return points

In [122]:
section_break_points = breakPoints(pTags)

In [123]:
section_break_points

[{'title': '1.1.1.1. Scope of Division A',
  'starting index': 0,
  'ending index': 2},
 {'title': '1.1.1.2. Scope of Division B',
  'starting index': 2,
  'ending index': 4},
 {'title': '1.1.1.3. Scope of Division C',
  'starting index': 4,
  'ending index': 6},
 {'title': '1.1.1.4. Internal Cross-references',
  'starting index': 6,
  'ending index': 8},
 {'title': '1.1.2.1. Application of Parts 1, 7 and 12',
  'starting index': 8,
  'ending index': 11},
 {'title': '1.1.2.2. Application of Parts 3, 4, 5 and 6',
  'starting index': 11,
  'ending index': 5}]

In [124]:
def organize(list):
    
    sections = []
    
    for i in list:
        
        title = i["title"]
        start = i["starting index"] + 1
        end = i["ending index"]
        items = pTags[start:end]
        
        obj = {"title": title, "items": items}
        
        sections.append(obj)
        
    return sections    

In [129]:
sections = organize(section_break_points)

In [133]:
def sortStuff(list):
    
    for i in list:
        
        title = i["title"]
        items = i["items"]
        
        cleaned = []
        
        for x in items:
            
            organ(x)

In [134]:
sortStuff(sections)

KeyError: 'class'

In [130]:
sections

[{'title': '1.1.1.1. Scope of Division A',
  'items': [<p class="subsection-e"><b>  (1) </b>Division A contains compliance and application provisions and the <i>objectives </i>and <i>functional statements</i>
   of this Code.</p>]},
 {'title': '1.1.1.2. Scope of Division B',
  'items': [<p class="subsection-e"><b>  (1) </b>Division B contains the <i>acceptable solutions</i>
   of this Code.</p>]},
 {'title': '1.1.1.3. Scope of Division C',
  'items': [<p class="subsection-e"><b>  (1) </b>Division C contains the administrative provisions of this Code.</p>]},
 {'title': '1.1.1.4. Internal Cross-references',
  'items': [<p class="subsection-e"><b>  (1) </b>If a provision of this Code contains a reference to another provision of this Code but no Division is specified, both provisions are in the same Division of this Code.</p>]},
 {'title': '1.1.2.1. Application of Parts 1, 7 and 12',
  'items': [<p class="subsection-e"> <b>(1)</b> Part 1 of Division B applies to all <i>buildings</i>.</p>,


In [132]:
def organ(list):
    
    for i in list:
        
        if i.attrs['class'] == ['subsection-e']:
        
            #get real sentence name
            x = re.search("\(([^)]+)\)", formatText(i))
            x = x.group(0)
            print(x)
            sentence = x
            print(sentence)
        
        

In [ ]:
organ()

In [82]:
#putting each class in a dictionary

for i in range(len(pTags)):
    
    #check class and put in dictionary based on class
    if pTags[i].attrs['class'] == ['section-e']:
        
        #get real article name
        x = re.search("\d\.\d\.\d\.\d\.", str(pTags[i]))
        x = x.group(0)
        article = x
        
        #increase article index
        #article += 1
        
        #New dictionary and entry for each article
        p[f'Art{article}'] = {f'Article{article}':pTags[i]}
        continue
    
    elif pTags[i].attrs['class'] == ['subsection-e']:
        
        #get real sentence name
        x = re.search("\(([^)]+)\)", str(pTags[i]))
        x = x.group(0)
        print(x)
        sentence = x
        
        #create dictionary first time, named Sentences
        #had to add a dummy entry because it was creating a list and typing seems difficult for nested dictionaries
#         if x == '(1)':
#             p[f'Art{article}'][f'Sen{x}'] = {'Dummy' : 'Entry'}
        
        p[f'Art{article}'][f'Sen{x}'] = { f'Sentence{x}': pTags[i]}
        continue
        
    elif pTags[i].attrs['class'] == ['clause-e']:
        
        #get real clause name
        x = re.search("\(([^)]+)\)", str(pTags[i]))
        x = x.group(0)
        clause = x
        
#         if x == '(a)':
#              p[f'Art{article}'][f'Sen{sentence}'][f'Cla{x}'] = {'Dummy' : 'Entry'}
        
        #create dictionary first time, then put in new entries
        p[f'Art{article}'][f'Sen{sentence}'][f'Cla{x}'] = {f'Clause{x}': pTags[i]}
        continue
    
    elif pTags[i].attrs['class'] == ['subclause-e']:
        
        #get real subclause name
        x = re.search("\(([^)]+)\)", (pTags[i]))
        x = x.group(0)
        subclause = x
        
#         if x == '(i)':
#             p[f'Art{article}'][f'Sen{sentence}'][f'Cla{clause}'][f'SubC{x}'] = {'Dummy' : 'Entry'}
        
        #create dictionary first time, then put in new entries
        p[f'Art{article}'][f'Sen{sentence}'][f'Cla{clause}'][f'SubC{x}'] = {f'Subclause{x}' : pTags[i]}
        continue


(1)
(1)
(1)
(1)
(1)
(2)
(1)


TypeError: expected string or bytes-like object

In [83]:
#test
p['Art1.1.2.2.']['Sen(1)']['Cla(a)']['SubC(ii)']['Subclause(ii)']

KeyError: 'SubC(ii)'